In [53]:
import numpy as np
from numpy import convolve
from numpy import genfromtxt
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter, windows
from scipy.integrate import RK45
from bokeh.plotting import figure, output_notebook, show
from bokeh.layouts import row, column
from bokeh.io import push_notebook, curdoc, export_png
from bokeh.models import Range1d, Label, Span, LinearAxis, Legend
from bokeh.models.glyphs import Line
from bokeh.themes import Theme
from bokeh.models.tickers import FixedTicker
output_notebook()
import ipywidgets as widgets
# import pandas as pd
# import qgrid
import ipywidgets
from ipywidgets import IntProgress, HBox, VBox, GridBox, Layout, Tab, FloatText
import sys
sys.path.insert(0,'C:\\Users\\Alan\\Anaconda3\\lib\\site-packages')

Loading BokehJS ...

Parameters Determined by Fiber Geometry and Materials

In [2]:
style = {'description_width': 'initial'}
box_titles = ['Core Diameter (micron)', 'Cladding Diameter (micron)', 'Fiber Loss (dB/m)',
            'Wavelength-CW Pump (nm)', 'Power - CW Pump (W)',
            'Pulse Energy - Fund (mJ)', 'Pulse Width - Fund (ns)', 'Wavelength - Fund (nm)', 'Pulse Width - Seed (ps)',
            'Pulse Energy - Seed (micJ)', 'Raman Shift (cm^-1)', 'Raman Gain (1E-12 cm/W)',
            'Raman Gain Width (cm^-1)', '2nd Stokes Loss (dB/m)', 'Lumped 8th Stokes Loss (dB)','Fiber Length (meter)',
            'Refractive Index', 'Time Increment (psec)', '# of Regen Passes']
box_values = [25.,250.,0.,
            976., 100., 40, 15,
            1030., 100, 10,
            450, 5,
            300, 0.0, 0, 0.2,
            1.5, 1., 1]
box_steps = [1., 1., 0.1,
             10, 1, 10, 1,
             10, 10, 0.1,
             5, 1,
             5, 1, 0.1, 0.1,
             0.1, 1, 1]
items = [FloatText(value=box_values[i], description=box_titles[i], step=box_steps[i], style=style) for i in range (len(box_titles))]
for i in range (len(items)): items[i].layout.width='230px'
gb=GridBox(items, layout=Layout(grid_template_columns="repeat(3, 250px)"))
gb

GridBox(children=(FloatText(value=25.0, description='Core Diameter (micron)', layout=Layout(width='230px'), st…

In [67]:
# Assign Values
coreD = gb.children[0].value*1e-4 # Core diameter in cm
cladD = gb.children[1].value*1e-4 # Cladding diameter in cm
fiber_loss = gb.children[2].value/100/4.34 # fiber background loss in inverse cm
pump_wavelength = gb.children[3].value # pump wavelenth in nm
pump_power = gb.children[4].value # pump power in W
f_energy = gb.children[5].value*1e-3 # Energy of fundamenatl pulse in mJ
f_width = gb.children[6].value*1e-9 # FWHM of fundamenatl pulse in nS
fundamental_wavelength = gb.children[7].value # wavelength of fundamenatl in nm
FWHM1 = gb.children[8].value*1e-12 # FWHM of Stokes seed in sec
Es10 = gb.children[9].value*1e-6 # Stokes seed energy in J
rshift = gb.children[10].value # Raman shift in inverse cm
raman_gain = gb.children[11].value*1e-12 # Peak Raman Gain in cm/W
raman_width = gb.children[12].value # Raman Gain Width in inverse cm
stokes2_loss = gb.children[13].value/4.34/1e2 # 2nd Stokes Loss in inverse cm
stokes3_loss = 0
stokes4_loss = 0
stokes5_loss = 0
stokes6_loss = 0
stokes7_loss = 0
stokes8_loss = 0
stokes8_loss_lump = 10**(-gb.children[14].value/10.0) # Lumped 2nd Stokes loss
L = gb.children[15].value # Fiber length in m
nref = gb.children[16].value # Refractive index
delta_t = gb.children[17].value*1e-12  # Time increment in sec
Npass = int(gb.children[18].value) # Number of single passes in regenerative amplifier


# Print Wavelengths
print('Fundamental Wavelength (nm) = ','{:5.0f}'.format(fundamental_wavelength))
stokesw1 = 1e7/(-rshift+1e7/fundamental_wavelength) # Wavelength of 1st Stokes signal in nm
print('First Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw1))
stokesw2 = 1e7/(-rshift+1e7/stokesw1) # Wavelength of 2st Stokes signal in nm
print('Second Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw2))
stokesw3 = 1e7/(-rshift+1e7/stokesw2) # Wavelength of 3rd Stokes signal in nm
print('Third Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw3))
stokesw4 = 1e7/(-rshift+1e7/stokesw3) # Wavelength of 4th Stokes signal in nm
print('Fourth Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw4))
stokesw5 = 1e7/(-rshift+1e7/stokesw4) # Wavelength of 1st Stokes signal in nm
print('Fifth Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw5))
stokesw6 = 1e7/(-rshift+1e7/stokesw5) # Wavelength of 2st Stokes signal in nm
print('Sixth Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw6))
stokesw7 = 1e7/(-rshift+1e7/stokesw6) # Wavelength of 3rd Stokes signal in nm
print('Seventh Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw7))
stokesw8 = 1e7/(-rshift+1e7/stokesw7) # Wavelength of 4th Stokes signal in nm
print('Eighth Stokes Wavelength (nm) = ','{:5.0f}'.format(stokesw8))

Aeff = np.pi*(coreD/2)**2
raman_width_Hz = raman_width/(1e7/stokesw1)*(3e17/stokesw1) # Width of equivalent spontaneous Raman input in Hz

Fundamental Wavelength (nm) =   1030
First Stokes Wavelength (nm) =   1080
Second Stokes Wavelength (nm) =   1135
Third Stokes Wavelength (nm) =   1196
Fourth Stokes Wavelength (nm) =   1264
Fifth Stokes Wavelength (nm) =   1341
Sixth Stokes Wavelength (nm) =   1427
Seventh Stokes Wavelength (nm) =   1525
Eighth Stokes Wavelength (nm) =   1637


In [68]:
# Create Signals
c=2.99792458e8
vg = c/nref
delta_L = vg * delta_t

hnu_stokes1=1239.842/stokesw1*1.6022e-19
hnu_stokes2=1239.842/stokesw2*1.6022e-19
hnu_stokes3=1239.842/stokesw3*1.6022e-19
hnu_stokes4=1239.842/stokesw4*1.6022e-19
hnu_stokes5=1239.842/stokesw5*1.6022e-19
hnu_stokes6=1239.842/stokesw6*1.6022e-19
hnu_stokes7=1239.842/stokesw7*1.6022e-19
hnu_stokes8=1239.842/stokesw8*1.6022e-19

# Create Signals
MM=1.3
N=int(round((2*MM+1)*L/delta_L))+1
t=np.linspace(-MM*L/vg, (MM+1)*L/vg,N)
z=np.linspace(-MM*L,(MM+1)*L,N)
# tend=(MM+0.5)*L/vg
tend1=0
tend2=2*L/vg
tend3=L/vg
fund_forw=(f_energy/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/f_width)*np.exp(-4*np.log(2)*((t-tend1)/(f_width))**2)*2
fund_back=(f_energy/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/f_width)*np.exp(-4*np.log(2)*((t-tend2)/(f_width))**2)*2
I01=(Es10/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/FWHM1)
stokes1 = I01*np.exp(-4*np.log(2)*(t/(FWHM1))**2)+hnu_stokes1*raman_width_Hz/Aeff
# +hnu_stokes1*raman_width_Hz/Aeff
stokes2=np.zeros(N)
stokes3=np.zeros(N)
stokes4=np.zeros(N)
stokes5=np.zeros(N)
stokes6=np.zeros(N)
stokes7=np.zeros(N)
stokes8=np.zeros(N)
int_mask = np.ones(N)

In [69]:
# Setup Figure
pg=figure(plot_width=650, plot_height=400, y_axis_type="log")
pg.min_border_top = 10

label1=Label(x=400, y=310, x_units='screen', y_units='screen', text='1st Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label1a=Label(x=400, y=290, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label2=Label(x=400, y=270, x_units='screen', y_units='screen', text='2nd Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label2a=Label(x=400, y=250, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label3=Label(x=400, y=230, x_units='screen', y_units='screen', text='3rd Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label3a=Label(x=400, y=210, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label4=Label(x=400, y=190, x_units='screen', y_units='screen', text='4th Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label4a=Label(x=400, y=170, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label5=Label(x=400, y=150, x_units='screen', y_units='screen', text='5th Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label5a=Label(x=400, y=130, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label6=Label(x=400, y=110, x_units='screen', y_units='screen', text='6th Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label6a=Label(x=400, y=90, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label7=Label(x=400, y=70, x_units='screen', y_units='screen', text='7th Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label7a=Label(x=400, y=50, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')
label8=Label(x=400, y=30, x_units='screen', y_units='screen', text='8th Stokes @ Output',
             text_font_size='10pt', text_color='darkblue')
label8a=Label(x=400, y=10, x_units='screen', y_units='screen', text='',
                 text_font_size='10pt', text_color='darkblue')

pg.x_range = Range1d(0,L)
pg.y_range = Range1d(1e-10,7e7)
cff=pg.line(z, fund_forw*Aeff, legend_label="Forward Fundamental",
           line_width=2, color='violet')
cfb=pg.line(z, fund_back*Aeff, legend_label="Backward Fundamental",
            line_width=2, line_dash='dashed', color='violet')
c1=pg.line(z, stokes1*Aeff, legend_label="Seed", line_width=2, color='blue')
c2=pg.line(z, stokes2*Aeff, legend_label="2nd Stokes", line_width=2, color='green')
c3=pg.line(z, stokes3*Aeff, legend_label="3rd Stokes", line_width=2, color='yellow')
c4=pg.line(z, stokes4*Aeff, legend_label="4th Stokes", line_width=2, color='orange')
c5=pg.line(z, stokes5*Aeff, legend_label="5th Stokes", line_width=2, color='red')
c6=pg.line(z, stokes6*Aeff, legend_label="6th Stokes", line_width=2, color='darkred')
c7=pg.line(z, stokes7*Aeff, legend_label="7th Stokes", line_width=2, color='brown')
c8=pg.line(z, stokes8*Aeff, legend_label="8th Stokes", line_width=2, color='black')
pg.legend.location = "top_left"; pg.legend.background_fill_alpha = 0.3
pg.xaxis.axis_label = 'Position (meter)'; pg.xaxis.axis_label_text_font_size = "12pt"
pg.xaxis.major_label_text_font_size = "12pt"
pg.yaxis.axis_label = 'Signal (Watt)'; pg.yaxis.axis_label_text_font_size = "12pt"
pg.yaxis.major_label_text_font_size = "12pt"

show(pg)

In [70]:
# Create Signals
MM=1.3
N=int(round((2*MM+1)*L/delta_L))+1
t=np.linspace(-MM*L/vg, (MM+1)*L/vg,N)
z=np.linspace(-MM*L,(MM+1)*L,N)
# tend=(MM+0.5)*L/vg
tend1=0
tend2=2*L/vg
tend3=L/vg
fund_forw=(f_energy/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/f_width)*np.exp(-4*np.log(2)*((t-tend1)/(f_width))**2)*2
fund_back=(f_energy/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/f_width)*np.exp(-4*np.log(2)*((t-tend2)/(f_width))**2)*2
I01=(Es10/Aeff)*(np.sqrt(4*np.log(2)/np.pi)/FWHM1)
stokes1 = I01*np.exp(-4*np.log(2)*(t/(FWHM1))**2)+hnu_stokes1*raman_width_Hz/Aeff
stokes2=np.zeros(N)
stokes3=np.zeros(N)
stokes4=np.zeros(N)
stokes5=np.zeros(N)
stokes6=np.zeros(N)
stokes7=np.zeros(N)
stokes8=np.zeros(N)

int_mask = np.zeros(N); int_mask[int(round(N/3)):int(round(2*N/3))]=1

nh=show(pg,notebook_handle='true');

import time
t0 = time.time()

M=1
delta_L=M*delta_L
P = IntProgress(min=0, max=int(round(N/(2*MM+1)/M)), description='Progress', bar_style='success'); display(P)
P2 = IntProgress(min=0, max=Npass, description='Progress', bar_style='success'); display(P2)

Eps = np.zeros([8,Npass+1])
Eps[0,0] = Es10

convsignal = windows.hamming(5)/np.sum(windows.hamming(5))

for k in range(Npass):
    if k % 2 == 0:
        fact=1
    else:
        fact=-1
    for i in range(int(round(N/(2*MM+1)/M))):
        fund_back[int(round((MM+1)*L/delta_L))]=fund_forw[int(round((MM+1)*L/delta_L))]
        fk1 = (-fund_forw*stokes1*raman_gain*stokesw1/fundamental_wavelength
                              - fund_forw*fiber_loss)*delta_L
        fbk1 = (-fund_back*stokes1*raman_gain*stokesw1/fundamental_wavelength
                              - fund_back*fiber_loss)*delta_L
        s1k1 = ((fund_forw+fund_back)*stokes1*raman_gain-stokes1*stokes2*raman_gain*stokesw2/stokesw1
                -stokes1*fiber_loss)*delta_L
        s2k1 = (stokes1*stokes2*raman_gain + stokes1*raman_gain*hnu_stokes2*raman_width_Hz/Aeff
                        -stokes2*stokes3*raman_gain*stokesw3/stokesw2
                        - stokes2*fiber_loss - stokes2*stokes2_loss)*delta_L  
        s3k1 = (stokes2*stokes3*raman_gain + stokes2*raman_gain*hnu_stokes3*raman_width_Hz/Aeff
                        -stokes3*stokes4*raman_gain*stokesw4/stokesw3
                         - stokes3*fiber_loss - stokes3*stokes3_loss)*delta_L
        s4k1 = (stokes3*stokes4*raman_gain + stokes3*raman_gain*hnu_stokes4*raman_width_Hz/Aeff
                        -stokes4*stokes5*raman_gain*stokesw5/stokesw4
                        - stokes4*fiber_loss - stokes4*stokes4_loss)*delta_L  
        s5k1 = (stokes4*stokes5*raman_gain + stokes4*raman_gain*hnu_stokes5*raman_width_Hz/Aeff
                        -stokes5*stokes6*raman_gain*stokesw6/stokesw5
                         - stokes5*fiber_loss - stokes5*stokes5_loss)*delta_L
        s6k1 = (stokes5*stokes6*raman_gain + stokes5*raman_gain*hnu_stokes6*raman_width_Hz/Aeff
                        -stokes6*stokes7*raman_gain*stokesw7/stokesw6
                        - stokes6*fiber_loss - stokes6*stokes6_loss)*delta_L  
        s7k1 = (stokes6*stokes7*raman_gain + stokes6*raman_gain*hnu_stokes7*raman_width_Hz/Aeff
                        -stokes7*stokes8*raman_gain*stokesw8/stokesw7
                         - stokes7*fiber_loss - stokes7*stokes7_loss)*delta_L
        s8k1 = (stokes7*stokes8*raman_gain + stokes7*raman_gain*hnu_stokes8*raman_width_Hz/Aeff
                         - stokes8*fiber_loss - stokes8*stokes8_loss)*delta_L
        
        fund_forw = fund_forw + (fk1)*int_mask
        fund_back = fund_back + (fbk1)*int_mask
        stokes1 = stokes1 + (s1k1)*int_mask
        stokes2 = stokes2 + (s2k1)*int_mask
        stokes3 = stokes3 + (s3k1)*int_mask
        stokes4 = stokes4 + (s4k1)*int_mask
        stokes5 = stokes5 + (s5k1)*int_mask
        stokes6 = stokes6 + (s6k1)*int_mask
        stokes7 = stokes7 + (s7k1)*int_mask
        stokes8 = stokes8 + (s8k1)*int_mask

        stokes1 = np.roll(stokes1,fact)
        stokes2 = np.roll(stokes2,fact)
        stokes3 = np.roll(stokes3,fact)
        stokes4 = np.roll(stokes4,fact)
        stokes5 = np.roll(stokes5,fact)
        stokes6 = np.roll(stokes6,fact)
        stokes7 = np.roll(stokes7,fact)
        stokes8 = np.roll(stokes8,fact)
        fund_forw = np.roll(fund_forw,1*fact)
        fund_back = np.roll(fund_back,-1*fact)

        P.value=i

        if i % int(N/(2*MM+1)/M/10) == 0 or i == int(round(N/(2*MM+1)/M))-1:
            cff.data_source.data['x']=t
            cff.data_source.data['y']=fund_forw*Aeff*np.heaviside(tend3-t,0)
            cfb.data_source.data['y']=fund_back*Aeff*np.heaviside(tend3-t,0)
            c1.data_source.data['y']=stokes1*Aeff
            c2.data_source.data['y']=stokes2*Aeff
            c3.data_source.data['y']=stokes3*Aeff
            c4.data_source.data['y']=stokes4*Aeff
            c5.data_source.data['y']=stokes5*Aeff
            c6.data_source.data['y']=stokes6*Aeff
            c7.data_source.data['y']=stokes7*Aeff
            c8.data_source.data['y']=stokes8*Aeff
            push_notebook(handle=nh)
    push_notebook(handle=nh)

    Eps[0,k+1]=np.sum(stokes1)*Aeff*delta_t*1e3
    Eps[1,k+1]=np.sum(stokes2)*Aeff*delta_t*1e3
    Eps[2,k+1]=np.sum(stokes3)*Aeff*delta_t*1e3
    Eps[3,k+1]=np.sum(stokes4)*Aeff*delta_t*1e3
    Eps[4,k+1]=np.sum(stokes5)*Aeff*delta_t*1e3
    Eps[5,k+1]=np.sum(stokes6)*Aeff*delta_t*1e3
    Eps[6,k+1]=np.sum(stokes7)*Aeff*delta_t*1e3
    Eps[7,k+1]=np.sum(stokes8)*Aeff*delta_t*1e3
    stokes8 = stokes8 * 1e-3
    P2.value=k+1
pg.add_layout(label1)
pg.add_layout(label1a)
pg.add_layout(label2)
pg.add_layout(label2a)
pg.add_layout(label3)
pg.add_layout(label3a)
pg.add_layout(label4)
pg.add_layout(label4a)
pg.add_layout(label5)
pg.add_layout(label5a)
pg.add_layout(label6)
pg.add_layout(label6a)
pg.add_layout(label7)
pg.add_layout(label7a)
pg.add_layout(label8)
pg.add_layout(label8a)
label1a.text='= {} mJ'.format("%2.3f" % Eps[0,-1])
label2a.text='= {} mJ'.format("%2.3f" % Eps[1,-1])
label3a.text='= {} mJ'.format("%2.3f" % Eps[2,-1])
label4a.text='= {} mJ'.format("%2.3f" % Eps[3,-1])
label5a.text='= {} mJ'.format("%2.3f" % Eps[4,-1])
label6a.text='= {} mJ'.format("%2.3f" % Eps[5,-1])
label7a.text='= {} mJ'.format("%2.3f" % Eps[6,-1])
label8a.text='= {} mJ'.format("%2.3f" % Eps[7,-1])


push_notebook(handle=nh)

"""
np.save('fundamental_1.npy', fundamental)
np.save('signal1_1.npy', signal1)
np.save('signal2_1.npy', signal2)
np.save('Aeff_1.npy', Aeff)
"""

t1=time.time()
print('elapsed time=',t1-t0, 'seconds')

IntProgress(value=0, bar_style='success', description='Progress', max=1001)

IntProgress(value=0, bar_style='success', description='Progress')

elapsed time= 660.7835457324982 seconds


In [71]:
# Save the Stokes energies
"""
for i in range(8):
    np.savetxt('Eps'+str(i)+',txt', Eps[i,:])
""""

SyntaxError: EOL while scanning string literal (Temp/ipykernel_6048/1258366691.py, line 5)

In [74]:
ep=figure(plot_width=650, plot_height=400)
ep.xaxis.axis_label = 'Number of Passes'; ep.xaxis.axis_label_text_font_size = "12pt"
ep.xaxis.major_label_text_font_size = "12pt"
ep.yaxis.axis_label = 'Pulse Energy (mJ)'; ep.yaxis.axis_label_text_font_size = "12pt"
ep.yaxis.major_label_text_font_size = "12pt"
ep.min_border_top = 10
ep.background_fill_color='#efefef'
ep.add_layout(Legend(), 'right')
ep.x_range = Range1d(0,60)
ep.y_range = Range1d(0,20)
ep1=ep.circle(np.arange(Npass+1),Eps[0,:],color='blue', legend_label="1st Stokes")
ep1a=ep.line(np.arange(Npass+1),Eps[0,:],color='blue', legend_label="1st Stokes")
ep2=ep.circle(np.arange(Npass+1),Eps[1,:],color='green', legend_label="2nd Stokes")
ep2a=ep.line(np.arange(Npass+1),Eps[1,:],color='green', legend_label="2nd Stokes")
ep3=ep.circle(np.arange(Npass+1),Eps[2,:],color='yellow', legend_label="3rd Stokes")
ep3a=ep.line(np.arange(Npass+1),Eps[2,:],color='yellow', legend_label="3rd Stokes")
ep3a=ep.line(np.arange(Npass+1),Eps[2,:],color='yellow', legend_label="3rd Stokes")
ep4=ep.circle(np.arange(Npass+1),Eps[3,:],color='orange', legend_label="4th Stokes")
ep4a=ep.line(np.arange(Npass+1),Eps[3,:],color='orange', legend_label="4th Stokes")
ep5=ep.circle(np.arange(Npass+1),Eps[4,:],color='red', legend_label="5th Stokes")
ep5a=ep.line(np.arange(Npass+1),Eps[4,:],color='red', legend_label="5th Stokes")
ep6=ep.circle(np.arange(Npass+1),Eps[5,:],color='darkred', legend_label="6th Stokes")
ep6a=ep.line(np.arange(Npass+1),Eps[5,:],color='darkred', legend_label="6th Stokes")
ep7=ep.circle(np.arange(Npass+1),Eps[6,:],color='brown', legend_label="7th Stokes")
ep7a=ep.line(np.arange(Npass+1),Eps[6,:],color='brown', legend_label="7th Stokes")
ep8=ep.circle(np.arange(Npass+1),Eps[7,:],color='black', legend_label="8th Stokes")
ep8a=ep.line(np.arange(Npass+1),Eps[7,:],color='black', legend_label="8th Stokes")

show(ep)

In [78]:
export_svg(ep, filename="ep_plot.svg")

NameError: name 'export_svg' is not defined

In [79]:
from bokeh.io import export_svg

export_svg( ep, filename="epp.svg")

['epp.svg']